In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb dill openpyxl torchmetrics
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover, lot_sizing
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst, with_lic


In [4]:
PRE_TRAIN_DIR = "temp/pre_train_tiny"
TRAIN_DIR = "temp/train_ls"
VALID_DIR = "temp/valid_ls"

In [5]:
import json
with open("gb.lic") as f:
    lic = json.load(f)

# parallel_generate_problem(lot_sizing, PRE_TRAIN_DIR, 20, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, TRAIN_DIR, 10000, 10)
# parallel_generate_solutions(TRAIN_DIR, 5, lic=lic)

# parallel_generate_problem(lot_sizing, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 5, lic=lic)
# assert 1 == 2

In [6]:
# import os
# import shutil

# # Define the source and destination folders
# src_folder = "temp/valid_ls"
# dest_folder = "temp/valid_ls_backup"

# # Create the destination folder if it doesn't exist
# os.makedirs(dest_folder, exist_ok=True)

# # List all files in the source folder
# files = os.listdir(src_folder)

# # First, process all .lp files: if a corresponding .npz file exists, move both to "done"
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".lp"):
#         lp_path = os.path.join(src_folder, file_name)
#         base_name = os.path.splitext(file_name)[0]
#         npz_file = base_name + ".npz"
#         npz_path = os.path.join(src_folder, npz_file)
        
#         # If the corresponding .npz exists, move both files
#         if os.path.exists(npz_path):
#             shutil.move(lp_path, os.path.join(dest_folder, file_name))
#             shutil.move(npz_path, os.path.join(dest_folder, npz_file))
#             print(f"Moved pair: {file_name} and {npz_file}")

# # Next, remove any .npz files that are left without a corresponding .lp file.
# for file_name in os.listdir(src_folder):
#     if file_name.endswith(".npz"):
#         base_name = os.path.splitext(file_name)[0]
#         lp_file = base_name + ".lp"
#         lp_path = os.path.join(src_folder, lp_file)
#         npz_path = os.path.join(src_folder, file_name)
        
#         if not os.path.exists(lp_path):
#             os.remove(npz_path)
#             print(f"Removed orphan npz: {file_name}")

In [7]:
# m = lot_sizing()
# model = m

# # model.setParam("OutputFlag", 0)
# model.setParam("PoolSolutions", 10)
# model.setParam("PoolSearchMode", 2)
# model = with_lic(model)
# model.optimize()

# vs = model.getVars()
# obj_val_and_sols = []

# for i in range(model.SolCount):
#     # TODO: setting solution number actually takes long time try to optimize
#     # TODO: add function to round by tolerance
#     model.params.SolutionNumber = i
#     obj_val = model.PoolObjVal
#     s = [obj_val] + [v.Xn for v in vs]
#     obj_val_and_sols.append(s)
# model.dispose()

# info = ModelInfo.from_model(m)
# info.var_info.sols = np.array(obj_val_and_sols)

In [8]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [9]:
# import os
# import pickle
# from temp.data.aug import augment_info, get_constraint_violations, get_lhs_matrix

# for p in os.listdir(os.path.join(PRE_TRAIN_DIR, "processed")):
#     if not p.endswith("pkl"):
#         continue
#     with open(os.path.join(PRE_TRAIN_DIR, "processed", p), "rb") as f:
#         info = pickle.load(f)

#     lhs = get_lhs_matrix(info.var_info.n, info.con_info)
#     rhs = np.array(info.con_info.rhs)
#     ops = np.array(info.con_info.types)
#     violations = get_constraint_violations(lhs, info.var_info.sols[0, 1:], rhs, ops)
#     assert not (np.abs(violations) >= 1e-5).any(), violations

    
#     # for _ in range(100):
#     #     aug_info = info.copy()
#     #     augment_info(aug_info)
#     #     lhs = get_lhs_matrix(aug_info.var_info.n, aug_info.con_info)
#     #     rhs = np.array(aug_info.con_info.rhs)
#     #     ops = np.array(aug_info.con_info.types)
#     #     violations = get_constraint_violations(lhs, aug_info.var_info.sols[0, 1:], rhs, ops)
#     #     assert not (np.abs(violations) >= 1e-5).any(), violations

In [10]:
from temp.data.graph import get_bipartite_graph, add_label
from temp.data.dataset import create_data_object

In [11]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info, force_reload=True)
# train_d = ModelGraphDataset(TRAIN_DIR, augment=augment_info)
# valid_d = ModelGraphDataset(VALID_DIR)

for i in range(len(pre_train_d)):
    data = pre_train_d[i][1]
    print(data.instance_name)

Processing...
(model, solution) => info:   0%|          | 0/20 [00:00<?, ?it/s]

(model, solution) => info: 100%|██████████| 20/20 [00:13<00:00,  1.50it/s]
Done!


aug_0_0_0
aug_0_0_1
>>> tensor(0.3635, dtype=torch.float64)
aug_1_1_0
aug_4_1_1
aug_0_2_0
>>> tensor(5.3893e-05, dtype=torch.float64)
aug_1_2_1
>>> tensor(0.0001, dtype=torch.float64)
aug_3_3_0
>>> tensor(0.0008, dtype=torch.float64)
aug_1_3_1
aug_3_4_0
aug_4_4_1
>>> tensor(0.0001, dtype=torch.float64)
aug_2_5_0
aug_0_5_1
>>> tensor(6.0111e-05, dtype=torch.float64)
aug_0_6_0
6_1
aug_3_7_0
aug_3_7_1
aug_3_8_0
>>> tensor(0.0002, dtype=torch.float64)
aug_0_8_1
>>> tensor(0.0004, dtype=torch.float64)
aug_0_9_0
>>> tensor(0.0023, dtype=torch.float64)
aug_0_9_1


In [13]:
np.hstack([[1], [2,3,4]])[:, np.newaxis]

array([1, 2, 3, 4])

In [12]:
assert 1 == 2

AssertionError: 

In [12]:
import os
import itertools

In [48]:
import itertools
import random
from typing import List

import gurobipy as gp
import numpy as np
import torch
from joblib import Parallel, delayed

from temp.data.info import ConInfo, ModelInfo, VarInfo

def get_lhs_matrix(n_var: int, con_info: ConInfo):
    shape = (con_info.n, n_var)

    row_idxs = [np.full(len(p), i, dtype=np.int64) for i, p in enumerate(con_info.lhs_p)]
    row_idxs = np.concatenate(row_idxs)
    col_idxs = np.concatenate(con_info.lhs_p)

    vals = np.concatenate(con_info.lhs_c)
    idxs = np.stack([row_idxs, col_idxs])
    lhs = torch.sparse_coo_tensor(idxs, vals, shape, dtype=torch.double)
    return lhs


def get_lhs_matrix(n_var: int, con_info: ConInfo) -> torch.Tensor:
    n_con = con_info.n
    shape = (n_con, n_var)

    idxs = [[], []]
    vals = []

    for con_idx in range(n_con):
        var_idxs = con_info.lhs_p[con_idx]
        var_cefs = con_info.lhs_c[con_idx]
        for var_idx, var_cef in zip(var_idxs, var_cefs):
            idxs[0].append(con_idx)
            idxs[1].append(var_idx)
            vals.append(var_cef)

    print(len(idxs[0]))
    print(len(set(zip(*idxs))))
    lhs = torch.sparse_coo_tensor(idxs, vals, shape)
    return lhs

In [49]:
info = os.info_py
m = get_lhs_matrix(info.var_info.n, info.con_info)
sorted(m.coalesce().values().numpy().tolist())

13966
12847


[-283.9822082519531,
 -266.23333740234375,
 -264.7542724609375,
 -248.4844512939453,
 -247.00537109375,
 -245.5262908935547,
 -244.04722595214844,
 -235.1727752685547,
 -217.42388916015625,
 -202.63314819335938,
 -198.1959228515625,
 -184.88426208496094,
 -183.40518188476562,
 -168.61444091796875,
 -162.69815063476562,
 -118.325927734375,
 -110.93055725097656,
 -110.31427764892578,
 -103.53518676757812,
 -102.91890716552734,
 -102.30261993408203,
 -101.68634033203125,
 -97.98865509033203,
 -90.5932846069336,
 -84.43048095703125,
 -82.58163452148438,
 -81.34907531738281,
 -77.03511047363281,
 -76.4188232421875,
 -70.25601959228516,
 -67.7908935546875,
 -49.302467346191406,
 -46.221065521240234,
 -45.96427917480469,
 -43.1396598815918,
 -42.882877349853516,
 -42.626094818115234,
 -42.36930847167969,
 -41.085391998291016,
 -40.82860565185547,
 -38.517555236816406,
 -38.303565979003906,
 -37.7472038269043,
 -35.9497184753418,
 -35.7357292175293,
 -35.52174377441406,
 -35.30775833129883,
 -

In [30]:

sorted(list(itertools.chain(*os.info_py.con_info.lhs_c)))

[-283.9822216627758,
 -266.2333328088523,
 -264.754258737692,
 -248.4844439549288,
 -247.0053698837686,
 -245.5262958126083,
 -244.047221741448,
 -235.1727773144862,
 -217.4238884605627,
 -202.6331477489597,
 -198.195925535479,
 -184.8842588950363,
 -183.4051848238761,
 -168.6144441122731,
 -162.698147827632,
 -118.3259256928233,
 -110.9305553370218,
 -110.3142744740383,
 -103.5351849812203,
 -102.9189041182369,
 -102.3026232552535,
 -101.68634239227,
 -97.98865721436927,
 -90.59328685856778,
 -84.43047822873324,
 -82.5816356397829,
 -81.34907391381599,
 -77.03510787293182,
 -76.41882700994836,
 -70.2560183801138,
 -67.79089492818,
 -49.30246903867636,
 -46.22106472375908,
 -45.9642810308493,
 -43.13966040884181,
 -42.88287671593205,
 -42.62609302302227,
 -42.3693093301125,
 -41.08539086556363,
 -40.82860717265386,
 -38.51755393646591,
 -38.30356752570776,
 -37.74720285773658,
 -35.94971700736817,
 -35.73573059661004,
 -35.52174418585189,
 -35.30775777509375,
 -35.17936592863885,
 -34.

In [15]:
c1 = np.sort(os.coeff_py.coalesce().values().numpy())

In [16]:
c2 = np.sort(os.coeff_th.flatten().numpy())

In [23]:
c1.tolist()

[-283.9822216627758,
 -266.2333328088523,
 -264.754258737692,
 -248.4844439549288,
 -247.0053698837686,
 -245.5262958126083,
 -244.047221741448,
 -235.1727773144862,
 -217.4238884605627,
 -202.6331477489597,
 -198.195925535479,
 -184.8842588950363,
 -183.4051848238761,
 -168.6144441122731,
 -162.698147827632,
 -118.3259256928233,
 -110.9305553370218,
 -110.3142744740383,
 -103.5351849812203,
 -102.9189041182369,
 -102.3026232552535,
 -101.68634239227,
 -97.98865721436927,
 -90.59328685856778,
 -84.43047822873324,
 -82.5816356397829,
 -81.34907391381599,
 -77.03510787293182,
 -76.41882700994836,
 -70.2560183801138,
 -67.79089492818,
 -49.30246903867636,
 -46.22106472375908,
 -45.9642810308493,
 -43.13966040884181,
 -42.88287671593205,
 -42.62609302302227,
 -42.3693093301125,
 -41.08539086556363,
 -40.82860717265386,
 -38.51755393646591,
 -38.30356752570776,
 -37.74720285773658,
 -35.94971700736817,
 -35.73573059661004,
 -35.52174418585189,
 -35.30775777509375,
 -35.17936592863885,
 -34.

In [28]:
graph = os.info_th
coeffs = []
for s, t, edge_data in graph.edges(data=True):
    coeffs.append(edge_data["coeff"])

In [29]:
sorted(coeffs)

[-283.9822216627758,
 -266.2333328088523,
 -264.754258737692,
 -248.4844439549288,
 -247.0053698837686,
 -245.5262958126083,
 -244.047221741448,
 -235.1727773144862,
 -217.4238884605627,
 -202.6331477489597,
 -198.195925535479,
 -184.8842588950363,
 -183.4051848238761,
 -168.6144441122731,
 -162.698147827632,
 -118.3259256928233,
 -110.9305553370218,
 -110.3142744740383,
 -103.5351849812203,
 -102.9189041182369,
 -102.3026232552535,
 -101.68634239227,
 -97.98865721436927,
 -90.59328685856778,
 -84.43047822873324,
 -82.5816356397829,
 -81.34907391381599,
 -77.03510787293182,
 -76.41882700994836,
 -70.2560183801138,
 -67.79089492818,
 -49.30246903867636,
 -46.22106472375908,
 -45.9642810308493,
 -43.13966040884181,
 -42.88287671593205,
 -42.62609302302227,
 -42.3693093301125,
 -41.08539086556363,
 -40.82860717265386,
 -38.51755393646591,
 -38.30356752570776,
 -37.74720285773658,
 -35.94971700736817,
 -35.73573059661004,
 -35.52174418585189,
 -35.30775777509375,
 -35.17936592863885,
 -34.

In [24]:
c2.tolist()

[-283.9822216627758,
 -266.2333328088523,
 -264.754258737692,
 -248.4844439549288,
 -247.0053698837686,
 -245.5262958126083,
 -244.047221741448,
 -235.1727773144862,
 -217.4238884605627,
 -202.6331477489597,
 -198.195925535479,
 -184.8842588950363,
 -183.4051848238761,
 -168.6144441122731,
 -162.698147827632,
 -118.3259256928233,
 -110.9305553370218,
 -110.3142744740383,
 -103.5351849812203,
 -102.9189041182369,
 -102.3026232552535,
 -101.68634239227,
 -97.98865721436927,
 -90.59328685856778,
 -84.43047822873324,
 -82.5816356397829,
 -81.34907391381599,
 -77.03510787293182,
 -76.41882700994836,
 -70.2560183801138,
 -67.79089492818,
 -49.30246903867636,
 -46.22106472375908,
 -45.9642810308493,
 -43.13966040884181,
 -42.88287671593205,
 -42.62609302302227,
 -42.3693093301125,
 -41.08539086556363,
 -40.82860717265386,
 -38.51755393646591,
 -38.30356752570776,
 -37.74720285773658,
 -35.94971700736817,
 -35.73573059661004,
 -35.52174418585189,
 -35.30775777509375,
 -35.17936592863885,
 -34.

In [ ]:
import os
coeff_diffs = np.sort(os.coeff_py.coalesce().values().numpy()) - np.sort(os.coeff_th.flatten().numpy())
sorted(coeff_diffs, reverse=True)

[0.9771301581422592,
 0.974,
 0.971412697677824,
 0.96672,
 0.96426587209728,
 0.9634082530276147,
 0.9584,
 0.9553323401216,
 0.946752,
 0.944165425152,
 0.9414532048441835,
 0.93020678144,
 0.9225943814045695,
 0.9190759441956863,
 0.9181963348934655,
 0.9148032,
 0.9146778976845823,
 0.9127584768,
 0.912,
 0.908,
 0.907,
 0.9032429767557119,
 0.903,
 0.8988449302446079,
 0.8977454186168319,
 0.8935672744312831,
 0.8933473721057279,
 0.8926876651290623,
 0.890948096,
 0.88736,
 0.8847711814090751,
 0.8830119628046335,
 0.88224,
 0.8821323535024127,
 0.88096,
 0.8790537209446398,
 0.879,
 0.878,
 0.8761510102473113,
 0.8759750883868671,
 0.87584,
 0.8750954790846464,
 0.8750954790846464,
 0.8742158697824255,
 0.8735561628057599,
 0.87218177327104,
 0.8715770418757631,
 0.8705215107130981,
 0.8691141358295448,
 0.8691141358295448,
 0.869,
 0.869,
 0.867,
 0.867,
 0.8669590930391039,
 0.8669590930391039,
 0.8666842151321599,
 0.8666842151321599,
 0.8662993860624383,
 0.8662993860624383,

In [17]:
(np.abs(coeff_diffs) > 1e-5).sum()

563

In [12]:
import os
n_added = len(os.rhs_py) if len(os.rhs_th) != len(os.rhs_py) else 0
os.rhs_th[-n_added:].squeeze() - os.rhs_py

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [13]:
sorted(os.diff_th.squeeze().numpy())

[-15951.119950908584,
 -8936.947924995735,
 -5898.939979545242,
 -4464.552453122335,
 -1486.5501888009728,
 -1389.4744387625685,
 -742.2538680006081,
 -454.38768281773605,
 -322.005767602176,
 -246.99244500025338,
 -246.46055625031676,
 -157.660842295296,
 -126.82449874944002,
 -84.69880078270467,
 -82.91646775295996,
 -78.41488306176,
 -70.83346467225593,
 -40.80593571839998,
 -32.7808370688,
 -29.062333985587145,
 -25.965153484799995,
 -16.09851469823998,
 -13.330187673599994,
 -7.0923059199999985,
 -6.9213644799999985,
 -6.799121408000002,
 -5.8207528959999975,
 -5.759959039999997,
 -3.8897791999999987,
 -3.359267840000001,
 -2.306048,
 -1.631232,
 -1.4855360000000006,
 -1.24416,
 -0.9021439999999998,
 -0.6784,
 -0.6612800000000008,
 -0.5595199999999991,
 -0.3360000000000001,
 -0.30739999999995776,
 -0.25060000000004207,
 -0.24636160000000107,
 -0.20240000000000002,
 -0.1442879999999993,
 -0.14100000000000001,
 -0.12149999999997887,
 -0.11540000000004225,
 -0.11039999999999994,
 -0.

In [14]:
sorted(os.diff_py)

[-15951.119453125002,
 -5898.939609375001,
 -1389.4743359375004,
 -454.3876806640626,
 -322.0057719824,
 -126.82449548827033,
 -84.6987866210938,
 -70.83347179201408,
 -32.78083616291954,
 -29.062342529296927,
 -16.098513850108603,
 -6.799121792149417,
 -3.359268160124513,
 -1.4855360496368415,
 -0.6612799618911751,
 -0.24636165391235476,
 -0.20239998984336854,
 -0.11540001392368726,
 -0.11040000724792476,
 -0.08820001745219791,
 -0.07000000321865082,
 -0.0009999859333249539,
 -4.00190824478841e-07,
 -2.1712793074613046e-07,
 -1.8093994391676915e-07,
 -7.271745705583754e-09,
 -2.9206276008775234e-09,
 -2.2737367544323206e-13,
 -8.881784197001252e-16,
 -4.440892098500626e-16,
 -1.1102230246251565e-16,
 -8.326672684688674e-17,
 -3.8163916471489756e-17,
 -3.122502256758253e-17,
 -2.7755575615628914e-17,
 -2.0816681711721685e-17,
 -1.3877787807814457e-17,
 -1.3877787807814457e-17,
 -6.938893903907228e-18,
 -6.938893903907228e-18,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


In [15]:
os.lhs_py.sort()
os.lhs_py

array([-1.59511195e+04, -5.89893961e+03, -1.38947434e+03, -4.54387681e+02,
       -3.22005772e+02, -1.26824495e+02, -8.46987866e+01, -7.08334718e+01,
       -3.27808362e+01, -2.90623425e+01, -1.60985139e+01, -6.79912179e+00,
       -3.35926816e+00, -1.48553605e+00, -6.61279962e-01, -2.46361654e-01,
       -2.02399990e-01, -1.15400014e-01, -1.10400007e-01, -8.82000175e-02,
       -7.00000032e-02, -9.99985933e-04, -4.00190824e-07, -2.17127931e-07,
       -1.80939944e-07, -7.27174571e-09, -2.92062760e-09, -2.27373675e-13,
       -8.88178420e-16, -4.44089210e-16, -1.11022302e-16,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [16]:
os.diff_py

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -2.27373675e-13,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [17]:
lhs_th = os.lhs_th.squeeze()[-n_added:]
lhs_th.sort()
lhs_th


array([-1.59511200e+04, -8.93694792e+03, -5.89893998e+03, -4.46455245e+03,
       -1.48655019e+03, -1.38947444e+03, -7.42253868e+02, -4.54387683e+02,
       -3.22005768e+02, -2.46992445e+02, -2.46460556e+02, -1.57660842e+02,
       -1.26824499e+02, -8.46988008e+01, -8.29164678e+01, -7.84148831e+01,
       -7.08334647e+01, -4.08059357e+01, -3.27808371e+01, -2.90623340e+01,
       -2.59651535e+01, -1.60985147e+01, -1.33301877e+01, -7.09230592e+00,
       -6.92136448e+00, -6.79912141e+00, -5.82075290e+00, -5.75995904e+00,
       -3.88977920e+00, -3.35926784e+00, -2.30604800e+00, -1.63123200e+00,
       -1.48553600e+00, -1.24416000e+00, -9.02144000e-01, -6.78400000e-01,
       -6.61280000e-01, -5.59520000e-01, -3.36000000e-01, -3.07400000e-01,
       -2.50600000e-01, -2.46361600e-01, -2.02400000e-01, -1.44288000e-01,
       -1.41000000e-01, -1.21500000e-01, -1.15400000e-01, -1.10400000e-01,
       -1.01500000e-01, -8.82000000e-02, -7.00000000e-02, -3.44000000e-02,
       -1.00000000e-03, -

In [18]:
sorted(os.coeff_th.squeeze())[-100:]

[tensor(2.8025, dtype=torch.float64),
 tensor(2.8697, dtype=torch.float64),
 tensor(2.8749, dtype=torch.float64),
 tensor(2.8991, dtype=torch.float64),
 tensor(2.9068, dtype=torch.float64),
 tensor(2.9439, dtype=torch.float64),
 tensor(2.9996, dtype=torch.float64),
 tensor(3.0305, dtype=torch.float64),
 tensor(3.1310, dtype=torch.float64),
 tensor(3.1851, dtype=torch.float64),
 tensor(3.2615, dtype=torch.float64),
 tensor(3.3243, dtype=torch.float64),
 tensor(3.3398, dtype=torch.float64),
 tensor(3.3843, dtype=torch.float64),
 tensor(3.4403, dtype=torch.float64),
 tensor(3.5872, dtype=torch.float64),
 tensor(3.6335, dtype=torch.float64),
 tensor(3.6799, dtype=torch.float64),
 tensor(3.7205, dtype=torch.float64),
 tensor(3.7447, dtype=torch.float64),
 tensor(3.7495, dtype=torch.float64),
 tensor(3.7882, dtype=torch.float64),
 tensor(3.8098, dtype=torch.float64),
 tensor(3.8345, dtype=torch.float64),
 tensor(3.8896, dtype=torch.float64),
 tensor(3.9187, dtype=torch.float64),
 tensor(3.98

In [19]:
sorted(os.coeff_py.coalesce().values())[-100:]

[tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dtype=torch.float64),
 tensor(1., dt

In [ ]:
assert 1 == 2

AssertionError: 

In [12]:

var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [13]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 16
CFG["num_layers"] = 8
CFG["hidden"] = 16

In [14]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [15]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [16]:
scheduler.total_steps = 10000

In [17]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, num_workers=8, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(pre_train_d, batch_size=2, shuffle=True, generator=torch.Generator().manual_seed(0))

In [18]:
scheduler.total_steps = 1000000

In [19]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/10 [00:00<?, ?it/s]

>>> tensor(0.0005)
>>> tensor(9105.7285)


 10%|█         | 1/10 [00:00<00:04,  2.06it/s]

>>> tensor(0.0002)
>>> tensor(85.6193)
>>> tensor(0.0005)
>>> tensor(3.4332e-05)


...

>>> tensor(1.1724)


  0%|          | 0/10 [00:00<?, ?it/s]


>> Epoch 1 ----------------------------------------------------------------------------------------------------
Training... 0


  0%|          | 0/10 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
assert 1 == 2

In [17]:
from temp.data.dataset import info_to_data

In [54]:
m = setcover(512, 512)

In [55]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [32]:
with torch.no_grad():
    logits = model(data)

In [33]:
binary_mask = torch.ones(len(logits), dtype=bool)

In [34]:
from temp.solver import upr

In [35]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [36]:
unc = upr.get_uncertainty(logits)

In [37]:
threshold = upr.get_threshold(unc)

In [38]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue

            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue

            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [39]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]


def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]
    bounds = fix_var(inst, conf_idxs, conf_vals)
    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs.tolist())
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [ ]:
indices = torch.arange(0, len(unc), dtype=int)
unc = unc.squeeze()
to_solve = m.copy()
to_solve.setParam("TimeLimit", 40)
to_solve.setParam("NoRelHeurTime", 20)
solve(to_solve, preds, unc, indices, 5)

In [ ]:
gb_m = m.copy()
gb_m.setParam("TimeLimit", 150)
gb_m.setParam("NoRelHeurTime", 60)
gb_m.optimize()

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def lot_sizing(
    n_periods: int = 20,
    n_items: int = 3,
    final_demand_low: int = 20,
    final_demand_high: int = 100,
    slack_low: int = 10,
    slack_high: int = 50,
    prod_cost_low: int = 1,
    prod_cost_high: int = 10,
    hold_cost_low: int = 1,
    hold_cost_high: int = 5,
    setup_cost_low: int = 50,
    setup_cost_high: int = 200,
    bom_ratio_low: int = 1,
    bom_ratio_high: int = 3,
    scale: float = 1e-3,
    cap_factor_final: float = 1.5,  # extra capacity multiplier for final product
    cap_factor: float = 1.2,        # extra capacity multiplier for intermediate items
    mo_factor: float = 0.8,         # multiplier to relax the minimum order requirement
    batch_levels: list = None,      # list of item indices that must be produced in batches
    batch_sizes: list = None,       # if provided as a list of length n_items, batch_sizes[i] is the batch size for item i
    seed: int = None,
) -> gp.Model:
    """
    Generate a multi-level capacitated lot-sizing instance with a BOM.
    
    The instance has n_items arranged in a chain:
      - Item (n_items-1) is the final product and receives random external demand.
      - Items 0,..., n_items-2 are intermediate items (with zero external demand).
    
    For each period t:
      - The final product's external demand is generated in [final_demand_low, final_demand_high].
      - For period t (before scaling) for the final product:
             cap_raw[t] = final_demand[t] + (final_demand[t+1] if t < n_periods-1 else 0) + slack_final[t],
             mo_raw[t]  = final_demand[t] + (final_demand[t+1] if t < n_periods-1 else 0).
        Then capacity for the final product is set as:
             capacity[t, n_items-1] = cap_raw[t] * cap_factor_final.
      - For lower items (i = n_items-2 downto 0), the capacity and minimum order are
             capacity[t, i] = bom_ratio[i+1] * capacity[t, i+1] * cap_factor,
             min_order[t, i] = bom_ratio[i+1] * min_order[t, i+1] * mo_factor.
      - The arrays demand, capacity, and min_order are then scaled by 'scale'.
    
    Decision variables for each period t and item i:
      - x[t,i]: production quantity (continuous, ≥ 0).
      - I[t,i]: inventory level at end of period t (continuous, ≥ 0).
      - y[t,i]: binary setup variable (1 if production is set up in period t for item i).
    
    Inventory balance:
      - For the final product (i = n_items-1):
            t = 0:  x[0, i] - I[0, i] = external demand[0]
            t ≥ 1:  I[t-1, i] + x[t, i] - I[t, i] = external demand[t]
      - For intermediate items (i = 0,..., n_items-2):
            t = 0:  x[0, i] - I[0, i] = bom_ratio[i+1] * x[0, i+1]
            t ≥ 1:  I[t-1, i] + x[t, i] - I[t, i] = bom_ratio[i+1] * x[t, i+1]
      - Final inventory is forced to 0.
    
    Production bounds:
      - x[t,i] ≤ capacity[t,i] * y[t,i]
      - x[t,i] ≥ min_order[t,i] * y[t,i]
    
    Batch Production:
      - For any item i that is in batch_levels, production must occur in multiples of a given batch size.
        That is, for each period t and such item i, an auxiliary integer variable z[t,i] is introduced and:
              x[t,i] = batch_size[i] * z[t,i].
    
    (No "no consecutive production" constraints are imposed here, so that the feasible region has room for multiple solutions.)
    
    Args:
        n_periods: Number of time periods.
        n_items: Number of items (BOM levels).
        final_demand_low, final_demand_high: Range for external demand for the final product.
        slack_low, slack_high: Range for additional slack (for the final product).
        prod_cost_low, prod_cost_high: Production cost per unit range (per item).
        hold_cost_low, hold_cost_high: Holding cost per unit range (per item).
        setup_cost_low, setup_cost_high: Fixed setup cost range (per item).
        bom_ratio_low, bom_ratio_high: Range for BOM ratios for items 1,..., n_items-1.
        scale: Scaling factor applied to demand, capacity, and minimum order.
        cap_factor_final: Multiplier to boost capacity of final product.
        cap_factor: Multiplier to boost capacity for intermediate items.
        mo_factor: Multiplier to relax the minimum order requirement.
        batch_levels: List of item indices (0 <= i < n_items) for which production is batched.
                      (Default is None, meaning no items are forced into batch production.)
        batch_sizes: If provided as a list (or array) of length n_items, then for any i in batch_levels,
                     x[t,i] must be an integer multiple of batch_sizes[i]. If not provided, for each i in
                     batch_levels a random batch size between 5 and 20 is generated.
        seed: Random seed for reproducibility.
        
    Returns:
        model: Gurobi model representing the instance.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # If no batch_levels provided, use empty list.
    if batch_levels is None:
        batch_levels = []
    
    # Process batch_sizes: if batch_levels is non-empty and batch_sizes is not provided, generate them.
    if batch_levels and batch_sizes is None:
        # Create a dictionary for batch sizes for each item in batch_levels.
        batch_sizes = {i: np.random.randint(5, 21) for i in batch_levels}
    elif batch_levels and isinstance(batch_sizes, list):
        # Assume batch_sizes is a list of length n_items.
        batch_sizes = {i: batch_sizes[i] for i in batch_levels}
    # Otherwise, assume batch_sizes is already a dictionary.
    
    # Generate external demand for final product (item index n_items-1)
    final_demand = np.random.randint(final_demand_low, final_demand_high + 1, size=n_periods)
    
    # Build demand matrix: only final product gets external demand.
    demand = np.zeros((n_periods, n_items), dtype=float)
    demand[:, n_items-1] = final_demand.astype(float)
    
    # Generate BOM ratios for items 1,..., n_items-1.
    bom_ratio = np.zeros(n_items, dtype=int)
    bom_ratio[0] = 0
    for i in range(1, n_items):
        bom_ratio[i] = np.random.randint(bom_ratio_low, bom_ratio_high + 1)
    
    # For final product, compute raw slack, capacity, and minimum order per period.
    slack_final = np.random.randint(slack_low, slack_high + 1, size=n_periods)
    capacity = np.empty((n_periods, n_items), dtype=float)
    min_order = np.empty((n_periods, n_items), dtype=float)
    for t in range(n_periods):
        next_demand = final_demand[t+1] if t < n_periods - 1 else 0
        cap_raw = final_demand[t] + next_demand + slack_final[t]
        mo_raw = final_demand[t] + next_demand
        capacity[t, n_items-1] = cap_raw * cap_factor_final
        min_order[t, n_items-1] = mo_raw
    
    # For lower levels, back-solve capacity and min_order.
    for i in range(n_items-2, -1, -1):
        for t in range(n_periods):
            capacity[t, i] = bom_ratio[i+1] * capacity[t, i+1] * cap_factor
            min_order[t, i] = bom_ratio[i+1] * min_order[t, i+1] * mo_factor
    
    # Apply scaling.
    demand *= scale
    capacity *= scale
    min_order *= scale
    
    # Generate cost parameters per item.
    prod_cost = np.random.randint(prod_cost_low, prod_cost_high + 1, size=n_items)
    hold_cost = np.random.randint(hold_cost_low, hold_cost_high + 1, size=n_items)
    setup_cost = np.random.randint(setup_cost_low, setup_cost_high + 1, size=n_items)
    
    model = gp.Model("lot_sizing")
    
    # Decision variables.
    # For every period and item, x[t,i] is production quantity.
    x = model.addVars(n_periods, n_items, lb=0, vtype=GRB.CONTINUOUS, name="x")
    I = model.addVars(n_periods, n_items, lb=0, vtype=GRB.CONTINUOUS, name="I")
    y = model.addVars(n_periods, n_items, vtype=GRB.BINARY, name="y")
    
    # For items that must be produced in batches, introduce auxiliary integer variable z.
    z = {}
    for i in batch_levels:
        for t in range(n_periods):
            z[t, i] = model.addVar(lb=0, vtype=GRB.INTEGER, name=f"z_{t}_{i}")
    
    model.update()
    
    # Linking constraints for batched items: x[t,i] = batch_size[i] * z[t,i]
    for i in batch_levels:
        for t in range(n_periods):
            model.addConstr(x[t, i] == batch_sizes[i] * z[t, i],
                            name=f"batch_link_{t}_item_{i}")
    
    # Inventory balance constraints.
    for i in range(n_items):
        for t in range(n_periods):
            if t == 0:
                if i == n_items - 1:
                    model.addConstr(x[0, i] - I[0, i] == demand[0, i],
                                    name=f"balance_0_item_{i}")
                else:
                    model.addConstr(x[0, i] - I[0, i] == bom_ratio[i+1] * x[0, i+1],
                                    name=f"balance_0_item_{i}")
            else:
                if i == n_items - 1:
                    model.addConstr(I[t-1, i] + x[t, i] - I[t, i] == demand[t, i],
                                    name=f"balance_{t}_item_{i}")
                else:
                    model.addConstr(I[t-1, i] + x[t, i] - I[t, i] == bom_ratio[i+1] * x[t, i+1],
                                    name=f"balance_{t}_item_{i}")
    
    # Final inventory is zero for every item.
    for i in range(n_items):
        model.addConstr(I[n_periods-1, i] == 0, name=f"final_inventory_item_{i}")
    
    # Capacity and minimum order constraints.
    for i in range(n_items):
        for t in range(n_periods):
            model.addConstr(x[t, i] <= capacity[t, i] * y[t, i],
                            name=f"cap_upper_{t}_item_{i}")
            model.addConstr(x[t, i] >= min_order[t, i] * y[t, i],
                            name=f"min_order_{t}_item_{i}")
    
    # (Optional) No consecutive production constraints are omitted to leave room for multiple solutions.
    
    # Objective: minimize total cost (production, holding, setup) over items and periods.
    model.setObjective(
        gp.quicksum(prod_cost[i] * x[t, i] + hold_cost[i] * I[t, i] + setup_cost[i] * y[t, i]
                    for t in range(n_periods) for i in range(n_items)),
        GRB.MINIMIZE
    )
    
    # Store instance data for later reference.
    model._demand = demand
    model._capacity = capacity
    model._min_order = min_order
    model._bom_ratio = bom_ratio
    model._prod_cost = prod_cost
    model._hold_cost = hold_cost
    model._setup_cost = setup_cost
    model._n_periods = n_periods
    model._n_items = n_items
    model._scale = scale
    if batch_levels:
        model._batch_levels = batch_levels
        model._batch_sizes = batch_sizes
    
    model.update()
    return model

In [ ]:
from temp.solver.utils import with_lic
m = lot_sizing(24 * 3 * 30 // 8, 24)
# m = with_lic(m)
m.setParam("NoRelHeurTime", 30)
m.optimize()

In [ ]:
24 * 3 * 30/8

In [13]:
from temp.data.info import ModelInfo
info = ModelInfo.from_model(m)

In [14]:
from temp.data.aug import augment_info

In [ ]:
new_info = augment_info(info)